In [1]:
%matplotlib widget
import bmcs_utils.api as bu
from bmcs_shear.shear_crack.crack_tip_orientation import CrackStateAnimator
ce = CrackStateAnimator()
ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
x_00=250
ce.sz_cp.trait_set(x_00=x_00, n_m = 15)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=300, B=200, L=500)
ce.sz_cp.add_x_tip_an([x_00-10, 50]) 
ce.sz_cp.add_x_tip_an([x_00-30, 100]) 
ce.sz_cp.sz_ctr.w = 10
#ce.sz_bd.cross_section_layout.reinforcement[0].trait_set(z=20, A=1, E=1)

In [2]:
ce.interact()

<lambdifygenerated-23>:2: RuntimeWarning: overflow encountered in exp
  return (select([less(f_c, -35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t**2*w),less_equal(w, 0.028*d_a**0.32*f_c**0.18/f_t),True], [-f_c,35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t**2*w,f_t*exp(-35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t*(-0.028*d_a**0.32*f_c**0.18/f_t + w))], default=nan))
<lambdifygenerated-24>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


In [5]:
%%capture
import numpy as np
import matplotlib.pylab as plt
from matplotlib import animation, rc

fig = plt.figure(figsize=(10,5))
fig.canvas.header_visible = False

psi_range = np.linspace(0, np.pi/4,4)
x_rot_1k = 240

def plot_stress_state(i):
    ce.psi_slider = psi_range[i]
    fig.clf()
    axes = ce.sz_stress_profile.subplots(fig)
    ce.sz_stress_profile.update_plot(axes)
    ax_u_0, ax_w_0, ax_S_0, ax_F_0, ax_u_1, ax_w_1, ax_S_1, ax_F_1 = axes
    
def plot_deformed_state(i):
    ce.psi_slider = psi_range[i]
    ce.x_rot_1k_slider = x_rot_1k
    fig.clf()
    axes = ce.sz_stress_profile.ds.subplots(fig)
    ce.sz_stress_profile.ds.update_plot(axes)
    axes.set_xlim(xmin=-50, xmax=550)
    axes.set_ylim(ymin=-30, ymax=350)

In [6]:
def html5_stress_state_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_stress_state,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

def html5_deformed_state_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_deformed_state,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

In [7]:
import os.path
home_dir = os.path.expanduser('~')
anim_dir = os.path.join(home_dir, 'generated_animation')
if not os.path.exists(anim_dir):
    os.makedirs(anim_dir)

In [8]:
%%capture
stress_anim_file = os.path.join(anim_dir, 'stress_state_animation.html')
with open(stress_anim_file,'w') as html_video_file:
    html_video_file.write(html5_stress_state_video())

In [9]:
%%capture
deformed_anim_file = os.path.join(anim_dir, 'deformed_state_animation.html')
with open(deformed_anim_file,'w') as html_video_file:
    html_video_file.write(html5_deformed_state_video())

In [10]:
from IPython.display import HTML
html_video_file = open(stress_anim_file,'r')
HTML(html_video_file.read())